In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd; pd.options.mode.chained_assignment = None

import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import plotly.express as px

import Models.Corn_USA_Yield as cy

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
scope = cy.Define_Scope()
w_df_all=uw.build_w_df_all(scope['geo_df'], scope['w_vars'], scope['geo_input_file'], scope['geo_output_column'])
w_df_all.keys()

dict_keys(['hist', 'gfs', 'ecmwf', 'gfsEn', 'ecmwfEn', 'hist_gfs', 'hist_ecmwf', 'hist_gfsEn', 'hist_ecmwfEn'])

In [6]:
w_df_all['gfs'][0:-1]

,IA_TempMax,IA_Prec,IL_TempMax,IL_Prec,IN_TempMax,IN_Prec,OH_TempMax,OH_Prec,MO_TempMax,MO_Prec,...,NE_TempMax,NE_Prec,IA_Sdd30,IL_Sdd30,IN_Sdd30,OH_Sdd30,MO_Sdd30,MN_Sdd30,SD_Sdd30,NE_Sdd30
time,,,,,,,,,,,,,,,,,,,,,
2022-07-10,29.33,0.00,27.26,0.02,26.41,0.00,26.34,0.00,30.33,0.00,...,33.72,0.00,0.00,0.00,0.00,0.00,0.33,0.00,1.65,3.72
2022-07-11,28.61,0.42,30.61,0.57,29.68,0.28,29.87,0.43,34.28,0.17,...,29.54,0.69,0.00,0.61,0.00,0.00,4.28,0.00,0.00,0.00
2022-07-12,27.91,0.09,27.88,0.70,27.17,1.44,27.50,2.69,30.27,1.01,...,30.36,0.03,0.00,0.00,0.00,0.00,0.27,0.00,0.00,0.36
2022-07-13,29.27,0.66,26.51,0.96,27.16,0.75,26.72,2.21,31.02,0.32,...,32.71,0.46,0.00,0.00,0.00,0.00,1.02,0.00,1.59,2.71
2022-07-14,29.96,0.54,27.00,0.04,25.82,0.03,26.03,0.21,31.86,0.05,...,34.87,0.14,0.00,0.00,0.00,0.00,1.86,0.00,5.61,4.87
2022-07-15,29.19,2.75,26.70,2.83,26.76,0.37,26.84,0.00,32.01,0.00,...,35.14,1.67,0.00,0.00,0.00,0.00,2.01,0.00,5.96,5.14
2022-07-16,29.54,3.39,30.19,5.91,28.63,0.81,28.95,0.05,34.05,0.52,...,32.04,0.52,0.00,0.19,0.00,0.00,4.05,0.00,0.00,2.04
2022-07-17,30.72,0.14,28.54,6.07,27.79,8.06,29.74,5.99,33.15,0.76,...,33.75,0.00,0.72,0.00,0.00,0.00,3.15,1.73,3.60,3.75
2022-07-18,31.40,0.01,30.01,0.20,27.88,0.84,27.41,3.78,34.50,0.60,...,36.03,0.01,1.40,0.01,0.00,0.00,4.50,2.55,6.89,6.03
